# Instigation in Lev 17-26

In [1]:
#Dataset path
PATH = 'datasets/'

from IPython.display import clear_output

import collections
import pandas as pd
import numpy as np

from tf.app import use

In [3]:
A = use('bhsa:local', hoist=globals(), mod='etcbc/heads/tf,''ch-jensen/semantic-roles/Aktionsart/tf')

Using TF-app in C:\Users\Ejer/text-fabric-data/annotation/app-bhsa/code:
	rv1.3=#f38d56bd757e87fe12d0c125e1ca52ee4376127b offline under ~/text-fabric-data (local release)
rate limit is 60 requests per hour, with 60 left for this hour


To increase the rate,see https://annotation.github.io/text-fabric/Api/Repo/


	connecting to online GitHub repo etcbc/bhsa ... connected
Using data in C:\Users\Ejer/text-fabric-data/etcbc/bhsa/tf/c:
	rv1.6=#bac4a9f5a2bbdede96ba6caea45e762fe88f88c5 (latest release)
rate limit is 60 requests per hour, with 55 left for this hour


To increase the rate,see https://annotation.github.io/text-fabric/Api/Repo/


	connecting to online GitHub repo etcbc/phono ... connected
Using data in C:\Users\Ejer/text-fabric-data/etcbc/phono/tf/c:
	r1.2=#1ac68e976ee4a7f23eb6bb4c6f401a033d0ec169 (latest release)
rate limit is 60 requests per hour, with 50 left for this hour


...

	connecting to online GitHub repo etcbc/parallels ... connected
Using data in C:\Users\Ejer/text-fabric-data/etcbc/parallels/tf/c:
	r1.2=#395dfe2cb69c261862fab9f0289e594a52121d5c (latest release)
rate limit is 60 requests per hour, with 45 left for this hour


To increase the rate,see https://annotation.github.io/text-fabric/Api/Repo/


	connecting to online GitHub repo etcbc/heads ... connected
Using data in C:\Users\Ejer/text-fabric-data/etcbc/heads/tf/c:
	rv.1.3.1=#6efbd7adb3ccfd3d4dcd780f76c5f86672395eb8 (latest release)
rate limit is 60 requests per hour, with 40 left for this hour


To increase the rate,see https://annotation.github.io/text-fabric/Api/Repo/


	connecting to online GitHub repo ch-jensen/semantic-roles ... connected
	downloading https://github.com/ch-jensen/semantic-roles/releases/download/1.7/Aktionsart-tf-c.zip ... 
	unzipping ... 
	saving data
Using data in C:\Users\Ejer/text-fabric-data/ch-jensen/semantic-roles/Aktionsart/tf/c:
	r1.7=#d02e3ec4bcd1b46a50b5033efc63010d0efac886 (latest release)
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used


In [4]:
all_verbs = '''
book book=Leviticus
  chapter chapter=17|18|19|20|21|22|23|24|25|26
    clause
       phrase function=Pred|PreO|PreS|PreC|PtcO
          word pdp=verb
'''

all_verbs = A.search(all_verbs)

  1.70s 936 results


## Active verbs

We can assume that active and causative verbs (in terms of aspect) are instigating. Stative verbs may be rendered dynamic by dynamic contexts. These need manual inspection as well as do passive verbs. We can therefore isolate those cases and extract the relevant phrase nodes for annotation.

In [6]:
passive = {'nif','pual','hof','ptcp'}

active_verbs = []
passive_verbs = []
causative_verbs = []
stative_verbs = []

for r in all_verbs:
    if (F.vs.v(r[4]) not in passive) and (F.vt.v(r[4]) not in passive): #passive verbs are excluded
        if F.dyna.v(r[4]) == 'sta' and not F.caus.v(r[4]): #Skipping verbs that are stative and not causative
            stative_verbs.append(r[2])
        elif F.caus.v(r[4]) == 'caus':
            causative_verbs.append(r[2])
        else:
            active_verbs.append(r[2]) #Appending all other verbs, that is, active, causative stative, and causative active.
    else:
        passive_verbs.append(r[2])
    
print(f'Number of active verbs: {len(active_verbs)}')
print(f'Number of stative verbs: {len(stative_verbs)}')
print(f'Number of causative verbs: {len(causative_verbs)}')
print(f'Number of passive verbs: {len(passive_verbs)}')

Number of active verbs: 316
Number of stative verbs: 186
Number of causative verbs: 350
Number of passive verbs: 84


These verbs should be annotated as instigating, either the predicate itself (implicit subject) or the explicit subject of the clause.

In [7]:
inst_lst = []

for cl in active_verbs:
    
    inst = int()
       
    phrases = L.d(cl, 'phrase')
    for ph in phrases:
        if F.function.v(ph) == 'Subj':
            inst = ph
            break
        elif F.function.v(ph) in {'Pred','PreS','PreO','PreC','PtcO'}:
            inst = ph
    
    inst_lst.append(inst)
    
len(inst_lst)

316

## Stative verbs

Stative verbs may be rendered dynamic by dynamic contexts, e.g. imperatives. We therefore need to inspect all stative verbs manually.

In [ ]:
anno = 'y n ?'.split()

STOP = 'stop'
error_message = "Input is invalid"

In [ ]:
def input_loop(question, right_answer):
    '''
    input_loop is used when the user is required to make an input. Question has to be a string,
    while right_answer is a list of accepted answer(s).
    '''
    while True:
        test = input(f'{question} ({",".join(right_answer)})') #The user is required to respond to message.
        if test in right_answer:
            return test
        elif test in [STOP]: #Input is accepted
            return test
        else: #Input is wrong and the user has respond again.
            print(error_message)

def get_head(ph):
    nom_head = E.nhead.t(ph)
    if len(nom_head) > 0:
        return nom_head[0]
    else:
        return E.head.t(ph)[0]

In [ ]:
def user_input(clauses):
    
    annotations = {}
    
    for clause_node in clauses:
    
        clear_output()
        
        Act = ''
        Act_Inst = ''

        #Defining possible Actor and Undergoer syntactic functions
        functions = ['Subj','Pred','PreS','PreO','PtcO','PreC','PtcO']

        test = ''

        for ph in L.d(clause_node, 'phrase'):

            head = get_head(ph) #Find nominal or prepositional head.

            #Annotate Actor
            if F.function.v(ph) in functions:

                #Visualize clause and highlight headword
                A.pretty(clause_node, highlights = {head: 'gold'})

                #1. test
                test = input_loop("Actor", anno)

                if test == 'y':
                    Act = ph

                    test = input_loop("Instigation", anno)

                    if test in anno:
                        Act_Inst = test

                    elif test == STOP: break

                elif test == '?':
                    Act = '?'
                    break

                elif test == 'n': continue

                elif test == STOP: break

        if test != STOP:
            
            comment = input('Make comment')
        
        if test == STOP:
            return annotations
        else:
            annotations[clause_node] = [Act, Act_Inst, comment]
            
    return annotations

In [ ]:
datasets = []

prev_runs = []

def Clauses(total_clauses, datasets, prev_runs):
        
    annotated_clauses = []
    remaining_clauses = []
    
    ##1. Check annotated datasets
    for csv in datasets:
        data = pd.read_csv(csv)
        annotated_clauses += list(data.iloc[:,0])
        
    ##2. Check previous runs (dictionaries with annotations)
    for r in prev_runs:
        for cl in r:
            annotated_clauses.append(cl)
            
    ##3. Determine the remaining clauses
    for cl in total_clauses:
        if not cl in annotated_clauses:
            remaining_clauses.append(cl)
    
    return remaining_clauses
        
remaining_clauses = Clauses(stative_verbs, datasets, prev_runs)
len(remaining_clauses)

In [ ]:
run6 = user_input(remaining_clauses)

In [ ]:
outputs = [run1, run2, run3, run4, run5, run6]

new_dict = {}
for r in outputs:
    for lex in r:
        new_dict[lex] = r[lex]
        
df = pd.DataFrame(new_dict).T
df.insert(0, 'clause', df.index)
df.columns = ['clause','actor','anno','comment']
df.head()

In [ ]:
df.to_csv(f'{PATH}Lev17-26.Instigation_stative_verbs.csv', index=None)

### Explore stative verbs in dynamic contexts

In [8]:
st_verbs = pd.read_csv(f'{PATH}Lev17-26.Instigation_stative_verbs.csv')

In [10]:
stative_verbs = st_verbs[st_verbs.anno == 'y']
stative_verbs = stative_verbs[['clause','actor','anno']]
stative_verbs.actor = stative_verbs.actor.astype('int') #Changing the phrase numbers to integers
stative_verbs.columns = ['clause','phr','Inst']

In [11]:
print(f'Number of stative verbs in dynamic contexts: {len(stative_verbs)}')

Number of stative verbs in dynamic contexts: 32


In [12]:
def getRefs(df):

    dic = collections.defaultdict(str)

    for n, row in df.iterrows():
        for w in L.d(row.clause, 'word'):
            if F.sp.v(w) == 'verb':
                bo, ch, ve = T.sectionFromNode(w)
                gloss = F.gloss.v(L.u(w, 'lex')[0])
                dic[f'{F.lex_utf8.v(w)} {F.vs.v(w)} {gloss}'] += f'{ch}:{ve}; '
            
    return dic

getRefs(stative_verbs)

defaultdict(str,
            {'שׁכב qal lie down': '18:22; 19:20; 20:11; 20:12; 20:13; 20:18; 20:20; ',
             'עמד qal stand': '18:23; ',
             'היה qal be': '19:2; 20:7; 20:26; 21:6; 21:6; ',
             'ירא qal fear': '19:3; 19:14; 19:30; 19:32; 25:17; 25:36; 25:43; 26:2; ',
             'שׂנא qal hate': '19:17; ',
             'נטר qal be angry': '19:18; ',
             'אהב qal love': '19:18; 19:34; ',
             'שׁבת qal cease': '23:32; ',
             'ישׁב qal sit': '23:42; 23:42; ',
             'שׁמע qal hear': '26:14; 26:18; 26:21; 26:27; '})

## Causative verbs

Causative verbs are most likely to involve an instigating causer. However, in some cases the the caused event may be occasioned by a non-intervening actor, e.g. something happens because the "causer" did not prevent it from happening, or it happened by accident. Therefore, we need to review all the causative events manually.

Only causatives with no instigating causer are annotated with instigation = "n"

In [ ]:
T.nodeFromSection(('Leviticus', 26,10))

In [ ]:
L.d(1417888, 'clause')

In [ ]:
datasets = []
prev_runs = []

caus_verbs1 = [cl for cl in causative_verbs if cl > 440690]

remaining_clauses = Clauses(caus_verbs1, datasets, prev_runs)
len(remaining_clauses)

In [ ]:
run1 = user_input(remaining_clauses)

### Explore causative verbs

In [13]:
caus_verbs = pd.read_csv(f'{PATH}Lev17-26.Instigation_causative_verbs.csv')
caus_verbs.head()

,clause,actor,anno
0,439655,688360,y
1,439658,688364,y
2,439659,688369,y
3,439660,688374,y
4,439661,688375,y


In [14]:
causative_verbs = caus_verbs[caus_verbs.anno == 'n']
causative_verbs = causative_verbs[['clause','actor','anno']]
causative_verbs.actor = causative_verbs.actor.astype('int') #Changing the phrase numbers to integers
causative_verbs.columns = ['clause','phr','Inst']

In [15]:
print(f'Number of causative verbs with a non-instigating causer: {len(causative_verbs)}')

Number of causative verbs with a non-instigating causer: 0


## Passive verbs

The passive verbs need to be manually inspected since they include reflexive and reciprocal meanings, both of which are instigating. Pure passives are not instigating, but they may be instigated by a complement phrase, expressing the Agent.

In [ ]:
remaining_clauses = Clauses(passive_verbs, datasets, prev_runs)
len(remaining_clauses)

In [ ]:
run1 = user_input(remaining_clauses)

### Merge outputs

In [ ]:
outputs = [run1]

new_dict = {}
for r in outputs:
    for lex in r:
        new_dict[lex] = r[lex]
        
df = pd.DataFrame(new_dict).T
df.insert(0, 'clause', df.index)
df.columns = ['clause','actor','anno','comment']
df.head()

In [ ]:
df.to_csv(f'{PATH}Lev17-26.Instigation_passive_verbs.csv')

### Review annotations

We review ambigious cases marked with '?':

In [16]:
df = pd.read_csv(f'{PATH}Lev17-26.Instigation_passive_verbs.csv')

In [17]:
ambig_cases = list(df[(df.anno == '?') | (df.comment == '?')].clause)
#ambig_cases

In [18]:
def show(cl):
    display(df[df.clause == cl])
    
    A.pretty(cl)

#### MKR "sell"

In [ ]:
show(ambig_cases[0])

While many translations and commentaries render this case (and the following three cases) reflexive, according to Milgrom this temptation "must be resisted" (Milgrom, p. 2219). A reflexive meaning is rendered in Deut 26:68 with Hitphael (see also 1 Kings for more examples) and all other cases of Niphal is passive. We therefore retain a passive use of the verb.

In [ ]:
MKR = '''
clause
  phrase function=Pred|PreO|PreS|PreC|PtcO
    word lex=MKR[ vs=nif|hit
'''

MKR = A.search(MKR)
A.show(MKR)

The other examples in Leviticus 25 are similar:

In [ ]:
show(ambig_cases[1])

In [ ]:
show(ambig_cases[2])

In [ ]:
show(ambig_cases[3])

#### KN< "be humble"

In Lev 26:41 the clause is frequently being translated "and then their uncircumcised heart will humble itself", but it could also simply be translated "is humbled" (NRSV) or "has become humbled". We will therefore retain the passive meaning.

In [ ]:
show(ambig_cases[4])

To sum up, no changes to the original annotation were made in this review, and we can continue to the final export.

In [19]:
df1 = df[df.anno == 'y']
df1 = df1[['clause','actor','anno']]
df1.actor = df1.actor.astype('int') #Changing the phrase numbers to integers
df1.columns = ['clause','phr','Inst']

In [20]:
print(f'Number of passive verbs with a reflexive/reciprocal meaning: {len(df1)}')

Number of passive verbs with a reflexive/reciprocal meaning: 10


In [21]:
getRefs(df1)

defaultdict(str,
            {'שׁבע nif swear': '19:12; ',
             'גרשׁ qal drive out': '21:7; ',
             'ענה pual be lowly': '23:29; ',
             'נצה nif decay': '24:10; ',
             'גאל nif redeem': '25:49; ',
             'חוה hsht bow down': '26:1; ',
             'יסר nif admonish': '26:23; ',
             'אסף nif gather': '26:25; ',
             'עזב nif leave': '26:43; ',
             'שׁמם hof be desolate': '26:43; '})

## Export

At this stage we should be ready to combine all annotations for both active and passive verbs. First, however, we will check for missing cases:

In [22]:
stative_verbs = pd.read_csv(f'{PATH}Lev17-26.Instigation_stative_verbs.csv')
stative_verbs.head()

,Unnamed: 0,clause,actor,anno,comment
0,439673,439673,NaN,NaN,NaN
1,439676,439676,NaN,NaN,NaN
2,439682,439682,NaN,NaN,NaN
3,439696,439696,NaN,NaN,NaN
4,439698,439698,NaN,NaN,NaN


In [23]:
remaining_verbs = set()

clause_list = [L.u(ph, 'clause')[0] for ph in inst_lst] #Changing phrase-nodes of inst_lst to clause nodes for comparison

for r in all_verbs:
    
    if r[2] in clause_list: #Check if clause exists in annotation of active verbs
        continue
        
    elif r[2] in list(stative_verbs.clause):
        continue
        
    elif r[2] in list(causative_verbs.clause):
        continue
        
    elif r[2] in list(df.clause): #Check if clause exists in annotation of passive verbs
        continue
        
    else:
        remaining_verbs.add(r[2])
        
print(f'Number of cases not annotated or reviewed: {len(remaining_verbs)}')

Number of cases not annotated or reviewed: 350


The remaining verbs are all causative verbs with an instigating actor.

There are no mismatches to review, so we can continue to the final export of this annotation. We therefore create a dataframe containing the data needed. We begin by adjusting the dataframe of stative verbs:

In [24]:
stative_verbs1 = stative_verbs[stative_verbs.anno == 'y']
stative_verbs1 = stative_verbs1[['clause','actor','anno']]
stative_verbs1.actor = stative_verbs1.actor.astype('int') #Changing the phrase numbers to integers
stative_verbs1.columns = ['clause','phr','Inst']

We do the same for the passive verbs dataset:

In [25]:
#df.head()

In [26]:
df1 = df[df.anno == 'y']
df1 = df1[['clause','actor','anno']]
df1.actor = df1.actor.astype('int') #Changing the phrase numbers to integers
df1.columns = ['clause','phr','Inst']

In [27]:
df1.head()

,clause,phr,Inst
14,439854,688937,y
34,440088,689652,y
49,440352,690433,y
51,440419,690645,y
66,440640,691335,y


And for the causative dataset:

In [28]:
causative_verbs1 = caus_verbs[caus_verbs.anno == 'y']
causative_verbs1 = causative_verbs1[['clause','actor','anno']]
causative_verbs1.actor = causative_verbs1.actor.astype('int') #Changing the phrase numbers to integers
causative_verbs1.columns = ['clause','phr','Inst']

In [29]:
causative_verbs1.head()

,clause,phr,Inst
0,439655,688360,y
1,439658,688364,y
2,439659,688369,y
3,439660,688374,y
4,439661,688375,y


Secondly, we transform the 'inst_lst' to a similar format by retrieving the clause node for each phrase:

In [30]:
inst_dict = {}

for ph in inst_lst:
    clause = L.u(ph, 'clause')[0]
    anno = 'y'
    
    inst_dict[clause] = [ph, anno]
    
df2 = pd.DataFrame(inst_dict).T
df2.insert(0, 'clause', df2.index)
df2.columns = ['clause','phr','Inst']
df2.head()

,clause,phr,Inst
439650,439650,688348,y
439651,439651,688350,y
439652,439652,688351,y
439653,439653,688354,y
439656,439656,688361,y


In [31]:
final_df = stative_verbs1.append(df1)
final_df = final_df.append(df2)
final_df = final_df.append(causative_verbs1)
final_df = final_df.sort_values(by='clause')
final_df.head()

,clause,phr,Inst
439650,439650,688348,y
439651,439651,688350,y
439652,439652,688351,y
439653,439653,688354,y
0,439655,688360,y


In [32]:
len(final_df)

708

In [33]:
final_df.to_csv(f'{PATH}Lev17-26.Instigation_final.csv', index=False)